In [11]:
import cugrad
a = cugrad.Tensor(2.0); a.label = 'a'
b = cugrad.Tensor(3.0); b.label = 'b'
c = cugrad.Tensor(1.0); c.label = 'c'
d = a * b; d.label = 'd'
e = d + c; e.label = 'e'
e.backward()

def dfs(t):
    print(t)
    for child in t.children:
        dfs(child)

dfs(e)

<Tensor data=2.000000, grad=0.000000, label=a>
